In [ ]:
!pip install timm==0.6.2dev0 wandb --upgrade

In [ ]:
import os
import pandas as pd
import wandb
from fastai.vision.all import *
from fastai.vision.learner import _update_first_layer
from fastai.callback.wandb import WandbCallback
import timm

# Testing Batch Sizes

Batch sizes don't need to be 32, 64 or 128. The fixation on powers of 2 for efficient GPU utilization might just be an urban myth.

This notebook explores the effect of different batch sizes on training runtimes.

# Setting up W&B

In [ ]:
# set your Weights & Biases Key in Kaggle via the menu Add-Ons > Secrets
from kaggle_secrets import UserSecretsClient
WANDB_API_KEY = UserSecretsClient().get_secret("WANDB_API_KEY")

In [ ]:
WANDB_PROJECT_NAME = 'Testing-Batch-Sizes'
wandb.login(key=WANDB_API_KEY)

# Loading Data

The data set consists of images of baby clothes for boys and girls.

In [ ]:
ROOT_IMAGE_PATH = '../input/hm-fashion-images-squared-224/images_224x224'

def image_path_for_article(article_id):
    id = f'{article_id:010d}'
    return(f'{ROOT_IMAGE_PATH}/{id[:3]}/{id}.jpg')   

In [ ]:
articles = pd.read_parquet('../input/hm-fashion-recommendation-parquet/articles.parquet')

In [ ]:
baby_boy_articles = articles[(articles.index_group=='Baby/Children') & (articles.department_name.str.contains('Baby Boy'))].article_id
baby_girl_articles = articles[(articles.index_group=='Baby/Children') & (articles.department_name.str.contains('Baby Girl'))].article_id

baby_girl_articles = list(filter(lambda article_id: os.path.isfile(image_path_for_article(article_id)), baby_girl_articles))
baby_boy_articles = list(filter(lambda article_id: os.path.isfile(image_path_for_article(article_id)), baby_boy_articles))

# Preparing Data Set

In [ ]:
article_imgs = DataBlock(blocks=(ImageBlock, CategoryBlock),
                 get_items=lambda source: baby_girl_articles + baby_boy_articles,
                 splitter=RandomSplitter(0.2),
                 get_y=lambda x: x in baby_girl_articles,
                 get_x=image_path_for_article,
                 item_tfms=None,
                 batch_tfms=aug_transforms(flip_vert=True) + [Normalize.from_stats(*imagenet_stats)])

In [ ]:
article_imgs.dataloaders(None, bs=16).show_batch(max_n=9, figsize=(6,7))

# Training

In [ ]:
def train(cfg, lr = None):
    wandb.init(project=WANDB_PROJECT_NAME, config=cfg)
    dls = article_imgs.dataloaders(None, bs=cfg['batch_size'])
    learn = vision_learner(dls, cfg['model_name'], pretrained=True, metrics=error_rate, cbs=WandbCallback()).to_fp16()
    if not 'lr' in cfg:
        lrs = learn.lr_find()
        cfg['lr'] = lrs.valley
        wandb.config.update(cfg)
    learn.fine_tune(cfg['epochs'], base_lr=cfg['lr'])
    wandb.config.update({"model": learn})
    wandb.finish()

In [ ]:
# create some learners just to download the model files
vision_learner(article_imgs.dataloaders(None, bs=1), 'convnext_base_in22k', pretrained=True, metrics=error_rate).to_fp16()
vision_learner(article_imgs.dataloaders(None, bs=1), 'deit_base_patch16_224', pretrained=True, metrics=error_rate).to_fp16()
vision_learner(article_imgs.dataloaders(None, bs=1), 'resnet50', pretrained=True, metrics=error_rate).to_fp16()

## ResNet50

In [ ]:
cfg={'batch_size': 128, 'model_name': 'resnet50', 'epochs': 80}
train(cfg)

In [ ]:
cfg={'batch_size': 129, 'model_name': 'resnet50', 'epochs': 80}
train(cfg)

## ConvNeXt

In [ ]:
cfg={'batch_size': 64, 'model_name': 'convnext_base_in22k', 'epochs': 80}
train(cfg)

In [ ]:
cfg={'batch_size': 65, 'model_name': 'convnext_base_in22k', 'epochs': 80}
train(cfg)

## DeiT

In [ ]:
cfg={'batch_size': 64, 'model_name': 'deit_base_patch16_224', 'epochs': 80}
train(cfg)

In [ ]:
cfg={'batch_size': 65, 'model_name': 'deit_base_patch16_224', 'epochs': 80}
train(cfg)

## Batch Sizes one by one

In [ ]:
for bs in range(8, 67):
    cfg={'batch_size': bs, 'model_name': 'convnext_base_in22k', 'epochs': 4, 'lr': 0.001}
    train(cfg)